In [1]:
from utils_optimized import *
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import roc_auc_score

In [2]:
train, test = load_data()
X_train, y_train = calculate_target(train)

In [16]:
X_tr, X_val, y_tr, y_val = train_test_split(X_train, y_train)

In [17]:
X_tr, X_val = add_features(X_tr, X_val, sort=True)

In [18]:
X_tr.columns

Index(['id', 'time', 'v_l', 'q', 'n_tr', 'code_azs', 'first_prch', 'location',
       'region', 'code', 'code1', 'type', 'date', 'sum_b', 'percent',
       'cur_points', 'full_month', 'first_prch_num', 'month', 'days',
       'time_weight', 'sum_b_tw', 'v_l_tw', 'q_tw', 'percent_tw'],
      dtype='object')

In [53]:
def aggregate_new(df, take_values=False):
    mode = lambda x: stats.mode(x).mode[0]
    fst = lambda vec: vec.iloc[0]
    simple_trend = lambda vec: np.sum(vec.shift(1)-vec[1:])
   
    num_features = ['min', 'max', 'median', 'sum', simple_trend]
    cat_features = [unique_cnt, 'min', 'max', mode]
   
    #'first_prch_num', 'q', 'v_l', 'month', 'time_weight', 'v_l_tw'
    res = df.groupby('id')[[ 'q_tw', 'sum_b', 'v_l_tw',
                            'percent_tw', 'time_weight']].agg({
#         'first_prch_num':'max',
#         'q':['sum', 'mean'],
        'q_tw':['sum', 'median', simple_trend],
        'v_l_tw':['sum', 'mean', simple_trend],
        'sum_b':['sum', 'mean', simple_trend],
#         'percent_tw': 'mean',
#         'first_purch_num': ['mean'],
#         'month':unique_cnt,
        'time_weight':['median', 'sum']
    })
 
    if take_values:
        return res.values, res.index
    else:
        return res

In [ ]:
X_tr_agg, X_val_agg = aggregate_new(X_tr), aggregate_new(X_val)

In [ ]:
knn = KNeighborsClassifier(100, weights='uniform', p=1)
y_pred = knn.fit(X_tr_agg, y_tr).predict_proba(X_val_agg)[:, 1]

In [52]:
roc_auc_score(y_val, y_pred)

0.7333501933092146

In [ ]:
roc_auc_score(y_val, y_pred)